In [1]:
import cv2
import tensorflow as tf
import numpy as np
from numpy import asarray

In [119]:
image = cv2.imread("images\word.jpg")

In [120]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 100,255 ,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [121]:
image.shape

(618, 1600, 3)

In [122]:
""" cv2.imshow("thresh", thresh)
cv2.waitKey(0) """

' cv2.imshow("thresh", thresh)\ncv2.waitKey(0) '

In [123]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 10))

In [124]:
""" cv2.imshow("ker", kernel)
cv2.waitKey(0) """

' cv2.imshow("ker", kernel)\ncv2.waitKey(0) '

In [125]:
dilate = cv2.dilate(thresh, kernel , iterations = 1)

In [126]:
""" cv2.imshow("di", dilate)
cv2.waitKey(0) """

' cv2.imshow("di", dilate)\ncv2.waitKey(0) '

In [127]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [128]:
len(cnts)

2

In [129]:
cnts =cnts[0] if len(cnts) == 2 else cnts[1]

In [130]:
cnts = sorted(cnts, key = lambda x : cv2.boundingRect(x)[0])

In [131]:
label_list= []
for i in range(97, 123):
    label_list.append(chr(i))

In [132]:
model=tf.keras.models.load_model("OCR_MODEL.h5")

In [133]:
ans =""

for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if h>50 and w >20 :
        roi = image[y:y+h+20, x:x+h+20]
        cv2.imshow("roi", roi)
        cv2.waitKey(0)
        cv2.rectangle(image, (x,y), (x+w, y+h), (36,255,12), 2)

        img=cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

        img = cv2.bitwise_not(img)

        img = cv2.resize(img, (28, 28))
        x = asarray(img)
        x = x.astype('float32')/255.0

        x = np.expand_dims(x, axis=0)
        
        x = np.where(x=='True', 1, x)
        x = np.where(x=='False', 0, x)
        
        print(type(x))
        print(x.shape)

        ocr_result = model.predict(x)

        print("highest accuracy " + str(ocr_result.max()))
        max = ocr_result.max()
        index = ocr_result.argmax()

        ans = ans + str((label_list[index]))
        

<class 'numpy.ndarray'>
(1, 28, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
highest accuracy 0.7781556
<class 'numpy.ndarray'>
(1, 28, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
highest accuracy 0.9997222
<class 'numpy.ndarray'>
(1, 28, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
highest accuracy 0.9999739
<class 'numpy.ndarray'>
(1, 28, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
highest accuracy 0.93591595
<class 'numpy.ndarray'>
(1, 28, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
highest accuracy 0.99171925


In [134]:
print(ans)

black
